In [16]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt;
import seaborn as sns;
from sklearn.utils import shuffle
from sklearn import svm
from sklearn import tree
from sklearn import neural_network
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
%matplotlib inline  


In [17]:
# ran from the root directory of the project folder from the zip file provided 
import os;
cwd = os.getcwd()
print(cwd)

/Users/owner/Desktop/asu/cse_572/Data_Mining_Assign1Data


In [18]:
def GetFilePath(cwd):
    emg=[]
    groundTruth=[]
    imu=[]
    
    for dirName, subdirList, fileList in os.walk(cwd):

        if 'Truth' in dirName and len(fileList)>0 and 'txt' in fileList[0]:
            groundTruth.append(dirName+'/'+fileList[0])

        MyoData=[(dirName,file) for file in fileList if 'EMG' in file]

        for f in MyoData:

            emg.append(f[0]+'/'+f[1])

        MyoData=[(dirName,file) for file in fileList if 'IMU' in file]

        for f in MyoData:

            imu.append(f[0]+'/'+f[1])
    return emg, groundTruth, imu
    
emg, groundTruth, imu= GetFilePath(cwd)

In [19]:
def indexingFilePath(groundTruth,emg):
    user_idx_groundTruth=[]
    user_idx_myo=[]
    matches=[]
    
    for i in range(len(groundTruth)):
        user_idx_groundTruth.append(groundTruth[i].split('/')[-3:-1])

        user_idx_myo.append(emg[i].split('/')[-3:-1])
    for i in range(len(user_idx_myo)):
        if user_idx_myo[i][0]=='user09':
            user_idx_myo[i][0]='user9'



    for i in range(len(user_idx_groundTruth)):

        for j in range (len(user_idx_myo)):

            if user_idx_groundTruth[i]==user_idx_myo[j]:
                matches.append((user_idx_groundTruth[i][0],user_idx_groundTruth[i][1],i,j))
    return matches

matches=indexingFilePath(groundTruth,emg)


In [20]:
def CreateGroundTruthDict(matches):
    groundTruth_dict={}
    groundTruth_col_names=['Start','End','unknown']
    for item in matches:

        user_id,user_utensil,groundTruth_idx,myo_idx =item
        df_name=user_id+'_'+user_utensil


        df=pd.read_csv(groundTruth[groundTruth_idx],
                       names=groundTruth_col_names)


        groundTruth_dict[df_name]=df

    # there was an error in user19_spoon data there was a ' ' instead of ',' so I was unable to do 100/30 and then
    # make then round and make it type int64
    groundTruth_dict['user19_spoon'].iloc[4,:]=[771,794,1]
    groundTruth_dict['user19_spoon']=groundTruth_dict['user19_spoon'].astype('int64')


    # multiply start and end rows by 100/30 so that you obtain the sample rows. Since python is 0
    # based so we also need to subtract 1 off this number to get the proper starting and ending rows indexes
    # if we were working in matlab we would not need to do this as it is 1 based. 

    for df in groundTruth_dict.values():
        df['Start']=df['Start']*100/30-1
        df['Start']=df['Start'].round()
        df['Start']=df['Start'].astype('int32')

        df['End']=df['End']*100/30
        df['End']=df['End'].round()
        df['End']=df['End'].astype('int32')
    return groundTruth_dict


groundTruth_dict=CreateGroundTruthDict(matches)


In [21]:
def CreateUserDict(file_list,matches,col_names):
    user_dict={}
    for item in matches:
        user_id,user_utensil,groundTruth_idx,myo_idx =item
        df_name=user_id+'_'+user_utensil
        df=pd.read_csv(file_list[myo_idx],names=col_names)
        user_dict[df_name]=df
    return user_dict

In [22]:
emg_col_names=['timestamp','EMG1','EMG2','EMG3','EMG4','EMG5','EMG6','EMG7','EMG8']
imu_col_names=['timestamp', 'Orientation X', 'Orientation Y', 'Orientation Z', 
               'Orientation W', 'Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 
               'Gyroscope X', 'Gyroscope Y','Gyroscope Z']

emg_dict=CreateUserDict(emg,matches,emg_col_names)
imu_dict=CreateUserDict(imu,matches,imu_col_names)




In [23]:
#function to put a users actions into either eating or non-eating catagory
def findActionsUser(groundTruth,myo,col_names):
    eating=pd.DataFrame(columns=col_names)
    non_eating=pd.DataFrame(columns=col_names)
    start_eating_idx=0
    stop_eating_idx=0
    
    for i in range(len(groundTruth)-1): 
        start_eating_idx=groundTruth.loc[i, "Start"]
        stop_eating_idx=groundTruth.loc[i, "End"]
        next_start_eating_idx=groundTruth.loc[i+1, "Start"]
        next_stop_eating_idx=groundTruth.loc[i+1, "End"]
        
        #print(start_eating_idx,stop_eating_idx)

        eating=eating.append(myo.iloc[start_eating_idx:stop_eating_idx,:])
        if i==0:
            non_eating=non_eating.append(myo.iloc[:start_eating_idx,:])
        else:
            non_eating=non_eating.append(myo.iloc[stop_eating_idx:next_start_eating_idx,:])
            
        #print(eating)
        #print(non_eating)
    eating=eating.append(myo.iloc[next_start_eating_idx:next_stop_eating_idx,:])
    non_eating=non_eating.append(myo.iloc[next_stop_eating_idx:,:])
    
    return eating, non_eating

In [24]:
def getAllActions(groundTruth_dict,myo_dict,col_names,random_sampling=True):
    eating=pd.DataFrame(columns=col_names)
    non_eating=pd.DataFrame(columns=col_names)
    user_action_dict={}
    for user_id,df in groundTruth_dict.items():

    
        eat_user,not_eat_user=findActionsUser(groundTruth_dict[user_id],myo_dict[user_id],col_names)
        eat_user.drop("timestamp", axis=1, inplace=True)
        not_eat_user.drop("timestamp", axis=1, inplace=True)
        

        #print(user_id)
        #create a dict with individual user actions 

        
        eat_user['class']='eat'
        not_eat_user['class']='not_eat'


        eating=eating.append(eat_user, sort=True)
        if random_sampling:
            non_eating=non_eating.append(not_eat_user.sample(len(eat_user),random_state=10),sort=True)
            user_action_dict[user_id]=[eat_user,not_eat_user.sample(len(eat_user),random_state=10)]
            
            
        else:
            non_eating=non_eating.append(not_eat_user, sort=True)
            user_action_dict[user_id]=[eat_user,not_eat_user]
           
                
    return eating,non_eating,user_action_dict


eat_emg,not_eat_emg,emg_action_dict=getAllActions(groundTruth_dict,emg_dict,emg_col_names)
eat_imu,not_eat_imu,imu_action_dict=getAllActions(groundTruth_dict,imu_dict,imu_col_names)



In [25]:
# reset indexes of all individual users actions as well as the emg and imu eat/not_eat dataframes

def reset_index(m):
    rang=range(0,len(m))
    m.index.name = None
    m.index=rang
    return

reset_index(eat_emg)
reset_index(not_eat_emg)
reset_index(eat_imu)
reset_index(not_eat_imu)

for k in emg_action_dict.keys():
    reset_index(emg_action_dict[k][0])
    reset_index(emg_action_dict[k][1])
    reset_index(imu_action_dict[k][0])
    reset_index(imu_action_dict[k][1])
    
  



In [26]:
def eat_not_eat_emg_statistics(eat,not_eat):
    
    not_eat_feat=not_eat.iloc[:,:-1]
    not_eat_statistics=pd.DataFrame()
    not_eat_statistics['mean'] = not_eat_feat.mean(axis=1)
    not_eat_statistics['rms'] =np.mean((not_eat_feat.iloc[:,:]**2),axis=1).pow(1/2)
    not_eat_statistics['std']=not_eat_feat.std(axis=1)
    not_eat_statistics['max']=not_eat_feat.max(axis=1)
    not_eat_statistics['min']=not_eat_feat.min(axis=1)
    not_eat_statistics['range']=not_eat_feat.max(axis=1)-not_eat_feat.min(axis=1)
    not_eat_statistics['variance']=not_eat_feat.var(axis=1)
    not_eat_statistics['cumsum']=not_eat_feat.abs().sum(axis=1)
    not_eat_statistics['eucladian norm']=np.sqrt(np.square(not_eat_feat).sum(axis=1))
    not_eat_statistics['class']='not eat'
    
    eat_feat=eat.iloc[:,:-1]
    eat_statistics=pd.DataFrame()
    eat_statistics['mean'] = eat_feat.mean(axis=1)
    eat_statistics['rms'] = np.mean((eat_feat.iloc[:,:]**2),axis=1).pow(1/2)
    eat_statistics['std']=eat_feat.std(axis=1)
    eat_statistics['max']=eat_feat.max(axis=1)
    eat_statistics['min']=eat_feat.min(axis=1)
    eat_statistics['range']=eat_feat.max(axis=1)-eat_feat.min(axis=1)
    eat_statistics['variance']=eat_feat.var(axis=1)
    eat_statistics['cumsum']=eat_feat.abs().sum(axis=1)
    eat_statistics['eucladian norm']=np.sqrt(np.square(eat_feat).sum(axis=1))
    eat_statistics['class']='eat'
    
    
    return eat_statistics, not_eat_statistics


In [27]:
# creating dictionarys containing each users statistical_feature_matrix of [eating,non_eating]

emg_action_dict_feature_matrix={}
imu_action_dict_feature_matrix={}

for k in emg_action_dict.keys():
    emg_eat,emg_no_eat=eat_not_eat_emg_statistics(emg_action_dict[k][0],emg_action_dict[k][1])
    emg_action_dict_feature_matrix[k]=[emg_eat,emg_no_eat]
    
    imu_eat,imu_no_eat=eat_not_eat_emg_statistics(imu_action_dict[k][0],imu_action_dict[k][1])
    imu_action_dict_feature_matrix[k]=[imu_eat,imu_no_eat]
emg_action_dict_feature_matrix



{'user39_fork': [       mean        rms        std   max   min  range    variance  cumsum  \
  0     5.500  11.618950  10.941402  30.0  -4.0   34.0  119.714286    58.0   
  1    -4.625   6.528782   4.926242   1.0 -14.0   15.0   24.267857    39.0   
  2     1.625   4.198214   4.138236   9.0  -3.0   12.0   17.125000    27.0   
  3    -3.375   7.591113   7.269064   4.0 -19.0   23.0   52.839286    41.0   
  4    -4.625   8.085481   7.089983   1.0 -20.0   21.0   50.267857    39.0   
  ...     ...        ...        ...   ...   ...    ...         ...     ...   
  2170 -0.250   1.224745   1.281740   1.0  -2.0    3.0    1.642857     8.0   
  2171  0.000   1.000000   1.069045   2.0  -1.0    3.0    1.142857     6.0   
  2172 -2.750   3.937004   3.011881   0.0  -9.0    9.0    9.071429    22.0   
  2173 -1.500   2.958040   2.725541   2.0  -6.0    8.0    7.428571    16.0   
  2174 -1.625   3.335416   3.113909   2.0  -8.0   10.0    9.696429    19.0   
  
        eucladian norm class  
  0          32

In [28]:
#because this is time series take aggregation to find the mean of each of the 
#5 statistical values over the amount of rows that allows we feed into row_merge. This allows us to have equal 
#number of eating and non-eating values between all the users. This can also be applied over time series that is 
#gathered in the future.



def row_merge(df_dict,n):
    aggregated_emg={}
    for k,v in df_dict.items():
        usr_eat, usr_not_eat=v[0],v[1]
        usr_len=v[0].shape[0]
        idx=np.linspace(0,usr_len,n)
        idx=idx.astype(int)
        df_eat=pd.DataFrame()
        df_not_eat=pd.DataFrame()



        for i in range(n-1):
            start=idx[i]
            stop=idx[i+1]
            df_eat=df_eat.append(usr_eat.iloc[start:stop,:-1].mean(axis=0), ignore_index=True)
            df_not_eat=df_not_eat.append(usr_not_eat.iloc[start:stop,:-1].mean(axis=0), ignore_index=True)
        df_eat['class']='eat'
        df_not_eat['class']='not eat'
        aggregated_emg[k]=[df_eat,df_not_eat]
    return aggregated_emg
    

        
        
aggregated_emg=row_merge(emg_action_dict_feature_matrix,50)
        
        
aggregated_emg

{'user39_fork': [       cumsum  eucladian norm        max      mean        min      range  \
  0   41.295455       20.015882  10.022727 -0.446023 -10.181818  20.204545   
  1   36.704545       17.300234   6.772727 -1.133523  -9.113636  15.886364   
  2   42.711111       20.473696   9.888889 -0.572222 -10.888889  20.777778   
  3   37.886364       18.148262   7.772727 -0.872159  -9.750000  17.522727   
  4   35.181818       15.699147   6.750000 -0.761364  -8.500000  15.250000   
  5   40.266667       19.983122   9.377778 -0.794444 -10.600000  19.977778   
  6   39.409091       20.703409  10.295455 -0.664773 -10.931818  21.227273   
  7   41.511111       21.453328   9.555556 -0.777778 -11.311111  20.866667   
  8   29.181818       14.146452   5.795455 -1.011364  -7.613636  13.409091   
  9   29.227273       13.918836   5.613636 -0.909091  -8.250000  13.863636   
  10  31.911111       14.478339   6.044444 -1.061111  -7.644444  13.688889   
  11  39.136364       19.206431   7.772727 -1.301

In [29]:
def append_df(df1,df2):
    return df1.append(df2, ignore_index=True)

merged_emg_feature_matrix={}
merged_aggregated_emg={}

for k,v in emg_action_dict_feature_matrix.items():
    merged_emg_feature_matrix[k]=append_df(emg_action_dict_feature_matrix[k][0],
                                           emg_action_dict_feature_matrix[k][1])
    
    merged_aggregated_emg[k]=append_df(aggregated_emg[k][0],
                                       aggregated_emg[k][1])
                                           
merged_emg_feature_matrix
merged_aggregated_emg
    

{'user39_fork':        cumsum  eucladian norm        max      mean        min      range  \
 0   41.295455       20.015882  10.022727 -0.446023 -10.181818  20.204545   
 1   36.704545       17.300234   6.772727 -1.133523  -9.113636  15.886364   
 2   42.711111       20.473696   9.888889 -0.572222 -10.888889  20.777778   
 3   37.886364       18.148262   7.772727 -0.872159  -9.750000  17.522727   
 4   35.181818       15.699147   6.750000 -0.761364  -8.500000  15.250000   
 ..        ...             ...        ...       ...        ...        ...   
 93  33.409091       16.856917   5.636364 -1.767045 -10.772727  16.409091   
 94  33.000000       15.062203   7.159091 -0.573864  -7.840909  15.000000   
 95  33.466667       15.580632   5.711111 -1.455556  -9.400000  15.111111   
 96  25.568182       11.547179   3.886364 -1.230114  -6.704545  10.590909   
 97  29.755556       14.012784   5.888889 -0.847222  -8.133333  14.022222   
 
          rms       std   variance    class  
 0   7.076683

In [30]:
type(list(emg_action_dict_feature_matrix.values())[0][0])


pandas.core.frame.DataFrame

In [31]:
#phase 1 train test split so that 60% of each user is in training set and 40% of each user is in testing set
# we will merge these together and apply pca over whole matrix then we will split them up again at the index 
# that represents the 60%. In doing this we will be able to run pca on all data together and then split it up
# at the appropriate point so as to gurantee that we have 60% of each users data in the training set and 40% of
#each users data in the test set.

#we have shuffled and stratified so that the samples are choosen randomly for test and train and so that 
# the test and train contain a 50/50 ratio of eat/no eat samples

def create_60_40_user_dependent_train_test(merged_emg_feature_matrix):
    from sklearn.model_selection import train_test_split
    
    train=pd.DataFrame()
    test=pd.DataFrame()

    for k,df in merged_emg_feature_matrix.items():
        
        X_user=df.iloc[:,:-1]
        y_user=df['class']

        X_train, X_test, y_train, y_test = train_test_split(X_user, y_user, test_size=0.4,train_size=0.6,random_state=42,
                                                            shuffle=True,stratify=y_user)
        train_user = pd.concat([X_train, y_train], axis=1)
        test_user= pd.concat([X_test, y_test], axis=1)
        
        train=train.append(train_user)
        test=test.append(test_user)

        
    return train,test ,train.shape[0] 

            
            
train_dependent,test_dependent, idx_60_percent_dependent=create_60_40_user_dependent_train_test(merged_aggregated_emg)



comined_train_test_dependent=append_df(train_dependent,test_dependent)
reset_index(comined_train_test_dependent)




comined_train_test_dependent



,cumsum,eucladian norm,max,mean,min,range,rms,std,variance,class
0,14.111111,6.613263,2.000000,-0.875000,-3.933333,5.933333,2.338141,1.975722,4.525000,eat
1,27.466667,12.897861,4.822222,-1.011111,-7.044444,11.866667,4.560082,3.925316,21.547619,not eat
2,31.911111,15.738303,5.511111,-1.394444,-9.533333,15.044444,5.564330,5.115841,38.113492,not eat
3,31.422222,14.963703,6.066667,-0.905556,-8.555556,14.622222,5.290468,4.751952,45.880952,not eat
4,34.227273,16.256433,6.659091,-1.227273,-8.318182,14.977273,5.747517,5.060495,39.657468,not eat
...,...,...,...,...,...,...,...,...,...,...
5875,46.681034,22.908316,11.224138,-0.490302,-10.982759,22.206897,8.099313,7.399185,77.999230,not eat
5876,39.704348,19.534198,9.165217,-0.689130,-10.260870,19.426087,6.906382,6.393742,52.722050,eat
5877,43.732759,21.289321,11.129310,-0.350216,-10.818966,21.948276,7.526912,7.141326,64.756927,not eat
5878,44.293103,21.595983,10.922414,-0.491379,-11.431034,22.353448,7.635333,7.269010,73.943042,not eat


In [32]:
train_dependent['class'].value_counts(), test_dependent['class'].value_counts()

(eat        1740
 not eat    1740
 Name: class, dtype: int64, eat        1200
 not eat    1200
 Name: class, dtype: int64)

In [33]:
# phase 2 we prepare our datasets so that we split the data so that in our training set we have 18 users spoon 
# and fork actions and in the test we have 12 different users spoon and fork actions. 
def create_60_40_user_independent_train_test(merged_emg_feature_matrix):
    from sklearn.model_selection import train_test_split
    
    train=pd.DataFrame()
    test=pd.DataFrame()
    count=0
    idx_60_percent=int(len(merged_emg_feature_matrix.keys())*.6)
    
    for k,df in merged_emg_feature_matrix.items():
        count+=1
        
        
        if count <=idx_60_percent:
            train=train.append(df)
            
        
        else:
            test=test.append(df)
        

        
    return train,test, train.shape[0] 


train_independent, test_independent, idx_60_percent_independent=create_60_40_user_independent_train_test(merged_aggregated_emg)

comined_train_test_independent=append_df(train_independent,test_independent)
reset_index(comined_train_test_independent)



comined_train_test_independent


,cumsum,eucladian norm,max,mean,min,range,rms,std,variance,class
0,41.295455,20.015882,10.022727,-0.446023,-10.181818,20.204545,7.076683,6.628677,51.266640,eat
1,36.704545,17.300234,6.772727,-1.133523,-9.113636,15.886364,6.116556,5.279620,31.351867,eat
2,42.711111,20.473696,9.888889,-0.572222,-10.888889,20.777778,7.238545,6.689990,51.000794,eat
3,37.886364,18.148262,7.772727,-0.872159,-9.750000,17.522727,6.416380,5.803764,40.260958,eat
4,35.181818,15.699147,6.750000,-0.761364,-8.500000,15.250000,5.550487,5.213100,32.000000,eat
...,...,...,...,...,...,...,...,...,...,...
5875,47.620690,23.521511,10.982759,-0.864224,-12.043103,23.025862,8.316110,7.628223,76.716133,not eat
5876,48.551724,24.053552,10.250000,-1.540948,-13.500000,23.750000,8.504215,7.838905,77.862685,not eat
5877,44.293103,21.595983,10.922414,-0.491379,-11.431034,22.353448,7.635333,7.269010,73.943042,not eat
5878,48.629310,23.830792,10.715517,-1.102371,-13.301724,24.017241,8.425457,7.869017,81.944119,not eat


In [34]:
train_independent['class'].value_counts(), test_independent['class'].value_counts()

(eat        1764
 not eat    1764
 Name: class, dtype: int64, eat        1176
 not eat    1176
 Name: class, dtype: int64)

In [35]:
# combine the train and test data and then run pca and split them based on the correct 60% idx point given

def pca(combined_df,split_idx):
    from sklearn import preprocessing
    from sklearn.decomposition import PCA
    
    label=combined_df['class'].copy()
    combined_df.drop("class", axis=1, inplace=True)
    
    # Get column names first
    names = combined_df.columns
    # Create the Scaler object
    scaler = preprocessing.StandardScaler()
    # Fit your data on the scaler object
    scaled_df = scaler.fit_transform(combined_df)
    scaled_df = pd.DataFrame(scaled_df, columns=names)
    
    
    pca = PCA(n_components=5)
    principalComponents = pca.fit_transform(scaled_df)


    pca_df = pd.DataFrame(data = principalComponents, columns = ['principal component 1',
                                                                 'principal component 2',
                                                                 'principal component 3',
                                                                 'principal component 4',
                                                                 'principal component 5'])
    
    pca_df['class']=label
    combined_df['class']=label
    

    train_pca=pca_df.iloc[:idx_60_percent_independent,:]
    test_pca=pca_df.iloc[idx_60_percent_independent:,:]
    
    return train_pca, test_pca

    
train_pca_independent, test_pca_independent =pca(comined_train_test_independent,idx_60_percent_independent)
train_pca_dependent, test_pca_dependent =pca(comined_train_test_dependent,idx_60_percent_dependent)

In [36]:
# merged dataset of all the users, we will use this for finding the optimal hyperparameters for all of all models 
# using sklearn GridsearchCV
def pca_nosplit(combined_df):
    from sklearn import preprocessing
    from sklearn.decomposition import PCA
    
    label=combined_df['class'].copy()
    combined_df.drop("class", axis=1, inplace=True)
    
    # Get column names first
    names = combined_df.columns
    # Create the Scaler object
    scaler = preprocessing.StandardScaler()
    # Fit your data on the scaler object
    scaled_df = scaler.fit_transform(combined_df)
    scaled_df = pd.DataFrame(scaled_df, columns=names)
    
    
    pca = PCA(n_components=5)
    principalComponents = pca.fit_transform(scaled_df)
    
    pca_df = pd.DataFrame(data = principalComponents, columns = ['principal component 1',
                                                                 'principal component 2',
                                                                 'principal component 3',
                                                                 'principal component 4',
                                                                 'principal component 5'])
    pca_df['class']=label
    combined_df['class']=label
    
    return pca_df
    
merged_dataset=pd.concat(list(merged_aggregated_emg.values()))
reset_index(merged_dataset)

pca_merged_dataset=pca_nosplit(merged_dataset)


In [37]:



df = shuffle(pca_merged_dataset,random_state=5)
X=df.iloc[:,:-1]
y=df.iloc[:,-1]



def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    kernel= ['linear', 'rbf']
    param_grid = {'C': Cs, 'gamma' : gammas, 'kernel':kernel}
    
    grid_search = GridSearchCV(svm.SVC(), param_grid, cv=nfolds, n_jobs=-1)
    grid_search.fit(X, y)


    return grid_search.best_params_,grid_search.cv_results_

def decisiontree_param_selection(X, y, nfolds): 
    criterion = ['gini', 'entropy']
    max_depth = [4,6,8,12]
    splitter=['best','random']
    
    param_grid={'criterion':criterion,  
                'max_depth': max_depth, 
                'splitter': splitter }


    grid_search = GridSearchCV(tree.DecisionTreeClassifier(), param_grid, cv=nfolds, n_jobs=-1)
    grid_search.fit(X, y)

    return grid_search.best_params_,grid_search.cv_results_

def rf_param_selection(X,y,nfolds):
    param_grid={'criterion': ['gini','entropy'],'max_depth':[4,6,8,10,12],
                'n_estimators': [3,10,100,300,1000] }
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=nfolds,n_jobs=-1)
    grid_search.fit(X, y)

    return grid_search.best_params_,grid_search.cv_results_

def nn_param_selection(X, y, nfolds): 
    param_grid= {'solver': ['lbfgs', 'sgd','adam'], 'max_iter': [500,1000,1500], 
                 'alpha': 10.0 ** -np.arange(1, 7), 'hidden_layer_sizes':np.arange(5, 12),
                 'activation': ['logistic', 'tanh','relu']}

    grid_search = GridSearchCV(neural_network.MLPClassifier(), param_grid, cv=nfolds,n_jobs=-1)
    grid_search.fit(X, y)

    return grid_search.best_params_,grid_search.cv_results_




decisiontree_best_perameters, cvres_decision_tree=decisiontree_param_selection(X, y, 5)
svm_best_perameters,cvres_svm=svc_param_selection(X, y, 5)
rf_best_perameters,cvres_svm=rf_param_selection(X, y, 5)


svm_best_perameters, decisiontree_best_perameters, rf_best_perameters

({'C': 10, 'gamma': 1, 'kernel': 'rbf'},
 {'criterion': 'gini', 'max_depth': 8, 'splitter': 'best'},
 {'criterion': 'gini', 'max_depth': 12, 'n_estimators': 100})

In [38]:
train_x_dependent, train_y_dependent = train_pca_dependent.iloc[:,:-1], train_pca_dependent.iloc[:,-1]
test_x_dependent,test_y_dependent =test_pca_dependent.iloc[:,:-1], test_pca_dependent.iloc[:,-1]

train_x_independent, train_y_independent = train_pca_independent.iloc[:,:-1], train_pca_independent.iloc[:,-1]
test_x_independent,test_y_independent =test_pca_independent.iloc[:,:-1], test_pca_independent.iloc[:,-1]

In [39]:
{'criterion': 'gini', 'max_depth': 12, 'n_estimators': 300}
rf_model_dependent = RandomForestClassifier(criterion='gini', max_depth=12, n_estimators=3000,random_state=10)
rf_model_dependent.fit(train_x_dependent, train_y_dependent)

rf_model_independent = RandomForestClassifier(criterion='gini',max_depth=12, n_estimators=3000,random_state=10)
rf_model_independent.fit(train_x_independent, train_y_independent)


svm_model_dependent=svm.SVC(C= 10, gamma= 1, kernel= 'rbf',random_state=10)
svm_model_dependent.fit(train_x_dependent, train_y_dependent)

svm_model_independent=svm.SVC(C= 10, gamma= 1, kernel= 'rbf',random_state=10)
svm_model_independent.fit(train_x_independent, train_y_independent)

decisiontree_model_dependent=tree.DecisionTreeClassifier(criterion='gini',max_depth=8, splitter='best',random_state=10)
decisiontree_model_dependent.fit(train_x_dependent, train_y_dependent)

decisiontree_model_independent=tree.DecisionTreeClassifier(criterion='gini',max_depth=8, splitter='best',random_state=10)
decisiontree_model_independent.fit(train_x_independent, train_y_independent)


nn_model_dependent=neural_network.MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=10,
                                                  max_iter=1500,solver= 'lbfgs',random_state=10)
nn_model_dependent.fit(train_x_dependent, train_y_dependent)

nn_model_independent=neural_network.MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=10,
                                                  max_iter=1500,solver= 'lbfgs',random_state=10)
nn_model_independent.fit(train_x_independent, train_y_independent)


print("RandomForests dependent-split Score: ",rf_model_dependent.score(test_x_dependent, test_y_dependent))
print("RandomForests independent-split Score: ",rf_model_independent.score(test_x_independent, test_y_independent))

print("SVM dependent-split Score: ",svm_model_dependent.score(test_x_dependent, test_y_dependent))
print("SVM independent-split Score: ",svm_model_independent.score(test_x_independent, test_y_independent))

print("DecisionTree dependent-split Score: ",decisiontree_model_dependent.score(test_x_dependent, test_y_dependent))
print("DecisionTree independent-split Score: ",decisiontree_model_independent.score(test_x_independent, test_y_independent))


print("neural network dependent-split Score: ",nn_model_dependent.score(test_x_dependent, test_y_dependent))
print("neural network independent-split Score: ",nn_model_independent.score(test_x_independent, test_y_independent))







RandomForests dependent-split Score:  0.8231292517006803
RandomForests independent-split Score:  0.7814625850340136
SVM dependent-split Score:  0.826530612244898
SVM independent-split Score:  0.7976190476190477
DecisionTree dependent-split Score:  0.7589285714285714
DecisionTree independent-split Score:  0.7725340136054422
neural network dependent-split Score:  0.8282312925170068
neural network independent-split Score:  0.8022959183673469


In [40]:
rf_y_trainpred_dependent=rf_model_dependent.predict(train_x_dependent)
rf_y_trainpred_independent=rf_model_independent.predict(train_x_independent)

svm_y_trainpred_dependent=svm_model_dependent.predict(train_x_dependent)
svm_y_trainpred_independent=svm_model_independent.predict(train_x_independent)

df_y_trainpred_dependent=decisiontree_model_dependent.predict(train_x_dependent)
df_y_trainpred_independent=decisiontree_model_independent.predict(train_x_independent)

nn_y_trainpred_dependent=nn_model_dependent.predict(train_x_dependent)
nn_y_trainpred_independent=nn_model_independent.predict(train_x_independent)

rf_y_testpred_dependent=rf_model_dependent.predict(test_x_dependent)
rf_y_testpred_independent=rf_model_independent.predict(test_x_independent)

svm_y_testpred_dependent=svm_model_dependent.predict(test_x_dependent)
svm_y_testpred_independent=svm_model_independent.predict(test_x_independent)

df_y_testpred_dependent=decisiontree_model_dependent.predict(test_x_dependent)
df_y_testpred_independent=decisiontree_model_independent.predict(test_x_independent)

nn_y_testpred_dependent=nn_model_dependent.predict(test_x_dependent)
nn_y_testpred_independent=nn_model_independent.predict(test_x_independent)


print('random forest train dependent')
print(classification_report(train_y_dependent, rf_y_trainpred_dependent, digits=4))
print('random forest test dependent')
print(classification_report(test_y_dependent, rf_y_testpred_dependent, digits=4))
print('random forest train independent')
print(classification_report(train_y_independent, rf_y_trainpred_independent, digits=4))
print('random forest test independent')
print(classification_report(test_y_independent, rf_y_testpred_independent, digits=4))

print('svm train dependent')
print(classification_report(train_y_dependent, svm_y_trainpred_dependent, digits=4))
print('svm test dependent')
print(classification_report(test_y_dependent, svm_y_testpred_dependent, digits=4))
print('svm train independent')
print(classification_report(train_y_independent, svm_y_trainpred_independent, digits=4))
print('svm test independent')
print(classification_report(test_y_independent, svm_y_testpred_independent, digits=4))

print('decision tree train dependent')
print(classification_report(train_y_dependent, df_y_trainpred_dependent, digits=4))
print('decision tree test dependent')
print(classification_report(test_y_dependent, df_y_testpred_dependent, digits=4))
print('decision tree train independent')
print(classification_report(train_y_independent, df_y_trainpred_independent, digits=4))
print('decision tree test independent')
print(classification_report(test_y_independent, df_y_testpred_independent, digits=4))

print('neural network train dependent')
print(classification_report(train_y_dependent, nn_y_trainpred_dependent, digits=4))
print('neural network test dependent')
print(classification_report(test_y_dependent, nn_y_testpred_dependent, digits=4))
print('neural network train independent')
print(classification_report(train_y_independent, nn_y_trainpred_independent, digits=4))
print('neural network test independent')
print(classification_report(test_y_independent, nn_y_testpred_independent, digits=4))










random forest train dependent
              precision    recall  f1-score   support

         eat     0.9476    0.9304    0.9389      1767
     not eat     0.9314    0.9483    0.9398      1761

    accuracy                         0.9393      3528
   macro avg     0.9395    0.9394    0.9393      3528
weighted avg     0.9395    0.9393    0.9393      3528

random forest test dependent
              precision    recall  f1-score   support

         eat     0.8221    0.8235    0.8228      1173
     not eat     0.8241    0.8227    0.8234      1179

    accuracy                         0.8231      2352
   macro avg     0.8231    0.8231    0.8231      2352
weighted avg     0.8231    0.8231    0.8231      2352

random forest train independent
              precision    recall  f1-score   support

         eat     0.9412    0.9257    0.9334      1764
     not eat     0.9269    0.9422    0.9345      1764

    accuracy                         0.9340      3528
   macro avg     0.9341    0.9340    